# Log Artfiacts and Import Models into Registries

Objective: various methods to bring existing models into Vertex Model Registry using Pre-built Containers and Custom Serving Containers. Uses user-supplied files including the dockerfile.

**Prereqs:**
* Create Artifact Registry repository for Docker images: https://cloud.google.com/artifact-registry/docs/repositories/create-repos#create
* Enable API's:
    * Vertex AI
    * Cloud Storage
    * Artifact Registry

**Resources:**
* [Import models to Vertex AI](https://cloud.google.com/vertex-ai/docs/model-registry/import-model#aiplatform_upload_model_sample-python_vertex_ai_sdk)

- `PROJECT_ID`: Google Cloud project ID where Vertex AI resources are deployed
- `REGION`: Google Cloud region to deploy Vertex AI resources.
- `VERSION`: Version tag for the Docker image.
- `REPO_NAME`: Name of the Artifact Registry repository for Docker images.
- `JOB_IMAGE_ID`: Name of the Docker image.
- `BUCKET_URI`: Google Cloud Storage bucket URI used for storing model artifacts and staging data.
- `model_file`: Name of the model file.
- `SERVICE_ACCOUNT`: Service account used to run Vertex AI jobs.

In [ ]:
# Image Parameters
PROJECT_ID = "sandbox-401718"  # @param {type:"string"}
REGION = "us-central1" # @param {type:"string"}
VERSION="latest" 
REPO_NAME="housing-poc" # @param {type:"string"}
JOB_IMAGE_ID="housing-poc-image" # @param {type:"string"}

# Cloud Storage 
BUCKET_URI = "gs://sandbox-401718-us-notebooks/gke-yaml"  # @param {type:"string"}
model_file = "model.pkl"

# Vertex Custom Job parameters
SERVICE_ACCOUNT="757654702990-compute@developer.gserviceaccount.com" # @param {type:"string"}

## Method 1: Import Model to Model Registry using Pre-built Containers

The most simple method which leverages pre-built container images by Google Cloud. These images support several common ML frameworks, including **Tensorflow, sklearn, PyTorch, and XGBoost.** The user just needs to supply model artifacts that comply with framework specific requirements to import the model into Vertex AI.


![method_1.png](./imgs/method_1.png)


List of pre-built containers: https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers#expandable-4

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

In [ ]:
# Save model to cloud storage to be imported into modle registry

! gsutil cp ./{model_file} {BUCKET_URI}/test/{model_file}

In [ ]:
# Import to model registry
DEPLOY_IMAGE_URI = "us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-5:latest"
display_model = "housing-poc-model"

vertex_model = aiplatform.Model.upload(
    display_name=display_model,
    serving_container_image_uri=DEPLOY_IMAGE_URI,
    artifact_uri=f"{BUCKET_URI}/test",
)

## Method 2: Import to Model Registry using Custom Serving Containers

This method requires users to be responsible for direct development of their own containers. This provides the most granular-level control, enabling users to provide the serving container rather than just the model artifact (see pre-built containers in Method 1). Users can customize the container to specific requirements such as model architectures, dependencies, and serving logic. [Custom Container Requirements](https://cloud.google.com/vertex-ai/docs/predictions/custom-container-requirements)

For custom containers on Vertex AI, model artifacts can either be bundled in the image (Method A) or decoupled by storing them separately in Cloud Storage (Method B)

In [ ]:
from google.cloud import aiplatform
# from google.cloud.aiplatform import explain

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

In [ ]:
# Build and push image to reigstry
! docker build -f ./Dockerfile -t {REGION}-docker.pkg.dev/{PROJECT_ID}/{REPO_NAME}/{JOB_IMAGE_ID}:{VERSION} .
! gcloud auth configure-docker us-central1-docker.pkg.dev --quiet
! docker push {REGION}-docker.pkg.dev/{PROJECT_ID}/{REPO_NAME}/{JOB_IMAGE_ID}:{VERSION}

### Method 2A: Containers with the model included

If your custom serving container image contains all the model artifacts required to serve predictions, you can register a Model in the Model Registry by providing the `serving container image` parameters.

![method_2a.png](./imgs/method_2a.png)


In [ ]:
vertex_model = aiplatform.Model.upload(
        display_name=display_model,
        serving_container_image_uri=f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPO_NAME}/{JOB_IMAGE_ID}:{VERSION}",
        serving_container_predict_route = "/predict",
        serving_container_health_route = "/health",
        serving_container_ports=[8080]
    )

### Method 2B: Containers without the model included

If model artifacts are stored separately in Cloud Storage, the container downloads them at startup using the AIP_STORAGE_URI environment variable provided by Vertex AI. Users can then register the model in the Model Registry by specifying the `custom serving image` along with the `artifact_uri` parameters.


Documenation for [Accessing Model Artifacts](https://cloud.google.com/vertex-ai/docs/predictions/custom-container-requirements#artifacts)

![method_2b.png](./imgs/method_2b.png)

Below is a possible example of a serving application that installs the model artifact into the serving container.

`main.py`

```{python} 
## Example serving application

import os
from typing import Annotated, get_type_hints

import joblib
from fastapi import FastAPI, HTTPException
from pandera import DataFrameModel
from pandera.typing import DataFrame as DataFrame
from pydantic import WithJsonSchema
import pandas as pd
import subprocess

# # Load your model
model = None
InputType = Annotated[DataFrame[DataFrameModel], WithJsonSchema(DataFrameModel.to_json_schema())]
OutputType = Annotated[DataFrame[DataFrameModel], WithJsonSchema(DataFrameModel.to_json_schema())]

app = FastAPI()
        
AIP_STORAGE_URI = os.environ.get('AIP_STORAGE_URI')
if AIP_STORAGE_URI:
    command = f"gcloud storage cp '{AIP_STORAGE_URI}/model.pkl' model.pkl"
    subprocess.run(command, shell=True, stdout=subprocess.PIPE)
    with open('model.pkl', 'rb') as f:
        model = joblib.load(f)
        types = get_type_hints(model.predict)
        r = types.pop('return')
        i = list(types.items())[0][1]
        InputType = Annotated[DataFrame[i], WithJsonSchema(i.to_json_schema())]
        OutputType = Annotated[DataFrame[r], WithJsonSchema(r.to_json_schema())]

@app.get("/health")
def health() -> dict[str, str]:
    if model is None:
        return {"STATUS": "ERROR", "MESSAGE": "Model not loaded"}
    return {"STATUS": "OK"}

@app.post("/predict/")
def predict(data: InputType) -> OutputType:
    return model.predict(pd.DataFrame(data))
```


In [ ]:
vertex_model = aiplatform.Model.upload(
        display_name=display_model,
        artifact_uri=f"{BUCKET_URI}/test",
        serving_container_image_uri=f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPO_NAME}/{JOB_IMAGE_ID}:{VERSION}",
        serving_container_predict_route = "/predict",
        serving_container_health_route = "/health",
        serving_container_ports=[8080]
    )

## Method 3: Custom Prediction Routines

Allows users to use the Vertex SDK to build custom containers with pre/post processing code. The SDK generates the Dockerfile and build images for the user. This offers a simplified interface for users to build and import models using containers by reducing the complexity of direct container development and deployment.


Sample Notebook: [SDK_Pytorch_Custom_Predict.ipynb](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/prediction/custom_prediction_routines/SDK_Pytorch_Custom_Predict.ipynb)